In [1]:
import sklearn
import numpy as np
import pickle
import pandas as pd

In [2]:
with open("./df_pickle/df_pool.pkl", "rb") as fp:
    df_pool = pickle.load(fp)

In [3]:
with open("./df_pickle/df_train.pkl", "rb") as fp:
    df_train = pickle.load(fp)

In [4]:
with open("./df_pickle/df_test.pkl", "rb") as fp:
    df_test = pickle.load(fp)

In [5]:
with open("./df_pickle/df_gen.pkl", "rb") as fp:
    df_gen = pickle.load(fp)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
import openai
import time
import os
import re

from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
class CommitteeMember:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Dense(128, input_dim=768*2, activation='relu'))
        self.model.add(Dense(2, activation='softmax'))

        # Compile the model
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train, epochs=25, batch_size=64)

In [8]:
neural_net = Sequential()
neural_net.add(Dense(256, input_dim=768*2, activation='relu'))
neural_net.add(Dense(256, activation='relu'))
neural_net.add(Dense(2, activation='softmax'))

neural_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
trainset = df_pool[df_pool['label'].notna()]['combined_embedding'].values
#pool = df_pool[df_pool['label'].isna()]['combined_embedding']
labels = df_pool[df_pool['label'].notna()]['label'].values
assert len(trainset) == len(labels)

In [10]:
df_test['combined_embedding'] = df_test.apply(lambda x: np.concatenate((x['question_embedding'],x['context_embedding'])), axis=1)
Xtest = df_test['combined_embedding'].values
ytest = df_test['label'].values

In [11]:
df_pool.loc[df_pool[df_pool['label'].isna()]['combined_embedding'].iloc[0:1].index]

,question,context,question_embedding,context_embedding,combined_embedding,combined_embedding_str,label
3698,What are ensemble methods and how do they work?,"A subspace of a vector space RM is a line, a ...","[tensor(0.2155), tensor(0.4005), tensor(-0.070...","[tensor(-0.0080), tensor(0.1167), tensor(0.482...","[0.21554105, 0.40045273, -0.070689894, 0.01691...",[ 0.21554105 0.40045273 -0.07068989 ... 0.02...,NaN


In [12]:
df_pool[df_pool['label'].isna()]['combined_embedding'].values

array([array([ 0.21554105,  0.40045273, -0.07068989, ...,  0.02724393,
               0.09465574,  0.6403501 ], dtype=float32)               ,
       array([ 0.31374073,  0.02718146, -0.08583552, ...,  0.02724393,
               0.09465574,  0.6403501 ], dtype=float32)               ,
       array([ 0.2254446 , -0.09242461,  0.12833361, ...,  0.02724393,
               0.09465574,  0.6403501 ], dtype=float32)               ,
       ...,
       array([-0.06985827, -0.05178229, -0.05998113, ..., -0.13981447,
               0.0680587 ,  0.11426937], dtype=float32)               ,
       array([-0.19451678, -0.10448775,  0.02188827, ..., -0.13981447,
               0.0680587 ,  0.11426937], dtype=float32)               ,
       array([ 0.01430442, -0.2677419 ,  0.14817478, ..., -0.13981447,
               0.0680587 ,  0.11426937], dtype=float32)               ],
      dtype=object)

In [13]:
df_pool

,question,context,question_embedding,context_embedding,combined_embedding,combined_embedding_str,label
0,﻿What is the difference between regression and...,How can we build intelligent machines? More t...,"[tensor(0.0604), tensor(-0.1970), tensor(-0.42...","[tensor(-0.1828), tensor(0.2544), tensor(-0.03...","[0.060423937, -0.19695072, -0.42494392, 0.1122...",[ 0.06042394 -0.19695072 -0.42494392 ... -0.15...,n
1,How is the decision tree algorithm modified to...,How can we build intelligent machines? More t...,"[tensor(-0.2524), tensor(-0.3376), tensor(-0.0...","[tensor(-0.1828), tensor(0.2544), tensor(-0.03...","[-0.2523584, -0.33764893, -0.08546509, 0.01299...",[-0.2523584 -0.33764893 -0.08546509 ... -0.15...,y
2,What is the stop criterion used in Hunt's algo...,How can we build intelligent machines? More t...,"[tensor(-0.1747), tensor(-0.4756), tensor(-0.1...","[tensor(-0.1828), tensor(0.2544), tensor(-0.03...","[-0.17472792, -0.47555625, -0.18331191, -0.080...",[-0.17472792 -0.47555625 -0.18331191 ... -0.15...,n
3,How is the impurity measure used in regression...,How can we build intelligent machines? More t...,"[tensor(-0.2113), tensor(-0.2685), tensor(-0.1...","[tensor(-0.1828), tensor(0.2544), tensor(-0.03...","[-0.21131495, -0.2685437, -0.14086674, 0.10631...",[-0.21131495 -0.2685437 -0.14086674 ... -0.15...,y
4,How does recursive binary splitting work in re...,How can we build intelligent machines? More t...,"[tensor(-0.1295), tensor(-0.2529), tensor(-0.3...","[tensor(-0.1828), tensor(0.2544), tensor(-0.03...","[-0.12945268, -0.25287953, -0.35721427, -0.131...",[-0.12945268 -0.25287953 -0.35721427 ... -0.15...,n
...,...,...,...,...,...,...,...
28315,How can we use the inverse cumulative density ...,The Apriori algorithm is very efficient at ide...,"[tensor(0.1509), tensor(-0.2756), tensor(-0.13...","[tensor(-0.3292), tensor(0.0511), tensor(0.269...","[0.15090862, -0.27562413, -0.13971977, -0.1023...",[ 0.15090862 -0.27562413 -0.13971977 ... -0.13...,NaN
28316,What are the two main steps when applying prob...,The Apriori algorithm is very efficient at ide...,"[tensor(0.0231), tensor(0.0890), tensor(-0.143...","[tensor(-0.3292), tensor(0.0511), tensor(0.269...","[0.023057327, 0.08897009, -0.14346834, 0.10672...",[ 0.02305733 0.08897009 -0.14346834 ... -0.13...,NaN
28317,What is anomaly detection and how is it relate...,The Apriori algorithm is very efficient at ide...,"[tensor(-0.0699), tensor(-0.0518), tensor(-0.0...","[tensor(-0.3292), tensor(0.0511), tensor(0.269...","[-0.06985827, -0.05178229, -0.059981134, 0.051...",[-0.06985827 -0.05178229 -0.05998113 ... -0.13...,NaN
28318,What are the limitations of the Gaussian mixtu...,The Apriori algorithm is very efficient at ide...,"[tensor(-0.1945), tensor(-0.1045), tensor(0.02...","[tensor(-0.3292), tensor(0.0511), tensor(0.269...","[-0.19451678, -0.10448775, 0.02188827, -0.0275...",[-0.19451678 -0.10448775 0.02188827 ... -0.13...,NaN


In [14]:
def oracle(idxs, df_pool):
    new_labels = []
    for i, idx in enumerate(idxs):
        ctx = df_pool.loc[idx]['context']
        q = df_pool.loc[idx]['question']

        #Check rate limit
        req_per_min += 1
        while req_per_min>=19:
            time_stamp = time.time()
            if int(time.time()-last_time_stamp) > 60:
                last_time_stamp = time_stamp
                req_per_min = 0
            else:
                time.sleep(10)
        
        #Ask the oracle for label for the context and two questions
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role":"system", "content":"You are a system designed to label if a list of provided questions can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `label: LABEL. LABEL should be 'y' if a question can be answered using the context and else 'n'"},
                {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a red book?```"},
                {"role":"assistant", "content":"label: y\nlabel: n"},
                {"role": "user", "content": f'CONTEXT: ```{ctx}``` QUESTION: ```{q["question"]}```'}
            ]
            )
        except Exception as e:
            print("OPENAI_ERROR:",str(e))
            continue
        
        #Parse the response to get labels
        res = completion.choices[0].message.content
        labels = re.findall(r'label: ([yn])', res)
        
        #If two labels are not found, assume something went wrong and skip this iteration
        if len(labels) != 1:
            print('[Error] labels not found in response:', res)
            continue
        
        new_labels.append(labels)

    return new_labels

In [16]:
#reset training set and pool
committee = [CommitteeMember() for _ in range(10)]
testacc_qbc=[] #this should hold the final accuracies
#Xtrain = trainset # should be the concatinated embeddings for question and context
#ytrain = labels
#Xpool = pool['combined_embedding'].values # unlabeled data in pool
n_samples = 50 # number of samples to be labeled from pool
n_add = 10
#poolidx=np.arange(len(Xpool),dtype=np.int64)
#poolidx=np.setdiff1d(poolidx,trainset)
for i in range(n_samples):
    print("run", i)
    Xtrain = df_pool[df_pool['label'].notna()]['combined_embedding'].values
    ytrain = df_pool[df_pool['label'].notna()]['label'].values
    Xpool = df_pool[df_pool['label'].isna()]['combined_embedding']
    ypool_lab = []
    # fit models in committee
    for j, committee_member in enumerate(committee):
        #bootstrapping
        Xtr,ytr=sklearn.utils.resample(Xtrain,ytrain,stratify=ytrain)
        #fit
        committee_member.train(Xtr.astype('float32'), ytr)
        #predict
        ypool_lab.append(committee_member.model.predict(Xpool.values))
        print(f"committee member {j} done")
    #get probability of label for each class based on voting in the committee
    ypool_p = np.mean(np.array(ypool_lab), 0)
    #Refit model in all training data
    neural_net.fit(Xtrain,ytrain)
    testacc_qbc.append((len(Xtrain),neural_net.evaluate(Xtest, ytest)))
    print(f"primaary model {i} done")
    #select sample with maximum disagreement (least confident)
    ypool_p_sort_idx = np.argsort(-ypool_p.max(1))[-n_add:] #least confident
    # make labels for new points
    new_labels = oracle(ypool_p_sort_idx, df_pool)
    #ypool = ['y']*10 #None
    df_pool.loc[Xpool[ypool_p_sort_idx].index]['label'] = new_labels
    #add to training set
    #Xtrain = np.concatenate((Xtrain,Xpool[ypool_p_sort_idx[-n_add:]]))
    #ytrain = np.concatenate((ytrain,ypool[ypool_p_sort_idx[-n_add:]]))
    #remove from pool
    #Xpool = np.delete(Xpool, ypool_p_sort_idx[-n_add:])
    #poolidx=np.setdiff1d(poolidx,poolidx[ypool_p_sort_idx[-n_add:]])
    print('Model: LR, %i samples (QBC)'%(n_samples+i*n_add))

run 0


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).